
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/monacofj/moeabench/blob/main/examples/example_10.ipynb)

# Example 10: Topological Equivalence (topo_dist)

This example demonstrates how to use the `topo_dist` tool to determine if two different algorithms have converged to statistically equivalent distributions in both the objective and decision spaces.

In [ ]:
!pip install --quiet git+https://github.com/monacofj/moeabench.git

In [ ]:
from MoeaBench import mb
print(f"Version: {mb.system.version()}")

# 1. Setup: Compare NSGA-II vs NSGA-III on DTLZ2
# We use a 3-objective problem (M=3)
exp1 = mb.experiment()
exp1.name = "NSGA-II"
exp1.mop = mb.mops.DTLZ2(M=3)
exp1.moea = mb.moeas.NSGA2(population=48, generations=50)

exp2 = mb.experiment()
exp2.name = "NSGA-III"
exp2.mop = mb.mops.DTLZ2(M=3)
exp2.moea = mb.moeas.NSGA3(population=48, generations=50)

# 2. Execution: Run both experiments
print("Running NSGA-II...")
exp1.run()

print("Running NSGA-III...")
exp2.run()

In [ ]:
# 3. Topological Analysis: Objective Space
# By default, topo_dist(exp1, exp2) uses space='objs' (the elite front)
print("\n--- [Analysis 1] Convergence Equivalence (Objective Space) ---")
res_objs = mb.stats.topo_distribution(exp1, exp2)
print(res_objs.report())

In [ ]:
# 4. Topological Analysis: Decision Space
# We can also check if they found the same solutions in the decision space
print("\n--- [Analysis 2] Strategy Equivalence (Decision Space) ---")
res_vars = mb.stats.topo_distribution(exp1, exp2, space='vars')
print(res_vars.report())

In [ ]:
# 5. Advanced: Earth Mover Distance (Geometric Distance)
# Quantify "how far" the distributions are from each other
print("\n--- [Analysis 3] Geometric Distance (EMD) ---")
res_emd = mb.stats.topo_distribution(exp1, exp2, method='emd')
print(f"Global Status (Matching): {res_emd.is_consistent}")
print(f"Failed Axes (Divergent): {res_emd.failed_axes}")

### 6. Visual Verification: Distribution Plots (topo_dist)

We now visualize the probability densities to intuitively understand the statistical verdicts. `topo_dist` combines the raw distribution with the $D$-statistic and $p$-value in the legend.

In [ ]:
# Visual 1: Density Comparison (Objective Space)
mb.view.topo_density(exp1, exp2, title="Comparison of Pareto Front Distributions")

In [ ]:
# Visual 2: Strategy Comparison (Decision Space - Axis 0)
mb.view.topo_density(exp1, exp2, space="vars", axes=[0], layout="independent")

### Interpretation

Unlike performance metrics (Hypervolume), which tell us "who is better", `topo_dist` tells us **"are they finding the same thing?"**.

1. **Objective Space Match**: If `topo_dist` reports a match in objective space, it means both algorithms provided a statistically similar approximation of the Pareto Front.

2. **Decision Space Divergence**: Sometimes, algorithms might reach the same quality (objectives) but through different sets of variables (multimodality). Divergence in `space='vars'` reveals these different search strategies.